In [53]:
import json
from collections import defaultdict

team_data_file = "/data/teams.json"
puzzle_data_file = "/data/puzzle-data.json"

In [54]:
teams = None
puzzles = None

with open(team_data_file, 'r') as f:
    teams = json.load(f)
    
with open(puzzle_data_file, 'r') as f:
    puzzles = json.load(f)
    
print("Loaded data for {teams} teams".format(teams=len(teams)))
print("Loaded data for {puzzles} puzzles".format(puzzles=len(puzzles)))

Loaded data for 76 teams
Loaded data for 4 puzzles


In [60]:
counts = defaultdict(lambda: {
    'team_count': 0,
    'teams_with_no_hints': 0,
    'teams_with_some_hints': 0,
    'teams_with_timeouts': 0,
    'hints_used': defaultdict(lambda: 0)
})

for team in teams:
    division = team.get('division')
    hints_used = team.get('hints_used')
    
    counts[division]['team_count'] += 1
    counts[division]['hints_used'][hints_used] += 1
    
    hint_count_key = 'teams_with_no_hints' if hints_used == 0 else 'teams_with_some_hints'
    counts[division][hint_count_key] += 1
    
    if team.get('timeouts') > 0:
        counts[division]['teams_with_timeouts'] += 1
    

for division in counts:
    div = counts[division]
    div['no_hints_percent'] = div['teams_with_no_hints'] / div['team_count']
    div['hints_percent'] = div['teams_with_some_hints'] / div['team_count']
    div['timeouts_percent'] = div['teams_with_timeouts'] / div['team_count']
    div['teams_with_no_timeouts'] = div['team_count'] - div['teams_with_timeouts']
    

In [61]:
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [62]:
# Pie charts for completion with and without hints by division

for division in counts:
    div = counts[division]
    
    hint_labels = ["No Hints", "Some Hints"]
    hint_values = [div['teams_with_no_hints'], div['teams_with_some_hints']]
    hint_domain = { 'x': [0, .48], 'y': [0,1] }
    
    timeout_labels = ["No Timeouts", "Some Timeouts"]
    timeout_values = [div['teams_with_no_timeouts'], div['teams_with_timeouts']]
    timeout_domain = { 'x': [.5, 1], 'y': [0,1] }
    
    hint_trace = go.Pie(
        labels=hint_labels,
        values=hint_values,
        domain=hint_domain
    )
    timeout_trace = go.Pie(
        labels=timeout_labels, 
        values=timeout_values, 
        domain=timeout_domain
    )
    
    layout = go.Layout(title=division)
    fig = go.Figure(data=[hint_trace, timeout_trace], layout=layout)
    iplot(fig)

In [63]:
# Bar Chart for Hints Taken by division

bar_chart_data = []

for division in counts:
    div = counts[division]
    trace = go.Bar(
        x=[num for num in range(13)],
        y=[div['hints_used'][num] for num in range(13)],
        name=division
    )
    
    bar_chart_data.append(trace)
    
layout = go.Layout(
    barmode='group',
    title='Hints Taken by Division',
    xaxis=dict(title="Number of hints taken"),
    yaxis=dict(title="Number of teams")
    
)
fig = go.Figure(data=data, layout=layout)

iplot(fig)


In [65]:
# Print Raw Data 
print(json.dumps(counts, indent=2))

{
  "wwu-alumni": {
    "teams_with_timeouts": 4,
    "hints_percent": 0.8,
    "timeouts_percent": 0.8,
    "teams_with_some_hints": 4,
    "hints_used": {
      "0": 1,
      "1": 0,
      "2": 1,
      "3": 2,
      "4": 1,
      "5": 0,
      "6": 0,
      "7": 0,
      "8": 0,
      "9": 0,
      "10": 0,
      "11": 0,
      "12": 0
    },
    "teams_with_no_timeouts": 1,
    "teams_with_no_hints": 1,
    "team_count": 5,
    "no_hints_percent": 0.2
  },
  "open": {
    "teams_with_timeouts": 32,
    "hints_percent": 0.85,
    "timeouts_percent": 0.8,
    "teams_with_some_hints": 34,
    "hints_used": {
      "0": 6,
      "1": 4,
      "2": 6,
      "3": 1,
      "4": 7,
      "5": 3,
      "6": 1,
      "7": 2,
      "8": 1,
      "9": 2,
      "10": 2,
      "11": 3,
      "12": 2
    },
    "teams_with_no_timeouts": 8,
    "teams_with_no_hints": 6,
    "team_count": 40,
    "no_hints_percent": 0.15
  },
  "wwu-student": {
    "teams_with_timeouts": 24,
    "hints_percent": 0.